In [ ]:
from argparse import ArgumentParser
import os.path as op
import matplotlib.pyplot as plt
import socket
import pandas as pd
import numpy as np
from time import sleep
import seaborn as sns

In [ ]:
%matplotlib inline

In [ ]:
# Generate base names
class DataFinder:
    def __init__(self, basedir, subject, run):
        self.basedir = basedir
        self.projectdir = op.join(
            basedir,
            "NIMH_SFIM",
            "handwerkerd",
            "ComplexMultiEcho1",
            "Data",
        )
        self.subject = subject
        self.run = run

    def set_subject(self, subject: int):
        self.subject = subject

    def set_run(self, run: int):
        self.run = run

    def subid(self):
        return f"sub-{self.subject:02}"

    def runid(self):
        return f"run{self.run:02}"

    def regressor_dir(self):
        return op.join(
            self.projectdir,
            self.subid(),
            "Regressors",
            "RejectedComps_c75",
        )

    def mixing_dir(self):
        sd = self.subid()
        return op.join(
            self.projectdir,
            sd,
            "afniproc_orig",
            "WNW",
            f"{sd}.results",
            f"tedana_c75_r{self.run:02}"
        )

    def regressor_prefix(self):
        subject = self.subject
        run = self.run
        return op.join(
            self.regressor_dir(),
            f"{self.subid()}_r{run:02}_CombinedRejected_c75_"
        )

    def combined_metrics(self):
        return self.regressor_prefix() + "Combined_Metrics.csv"

    def combined_betas(self):
        return self.regressor_prefix() + "betas.csv"

    def combined_r2(self):
        return self.regressor_prefix() + "R2vals.csv"

    def combined_f(self):
        return self.regressor_prefix() + "Fvals.csv"

    def combined_p(self):
        return self.regressor_prefix() + "pvals.csv"

    def full_model(self):
        return self.regressor_prefix() + "FullRegressorModel.csv"

    def mixing_matrix(self):
        return op.join(
            self.mixing_dir(),
            "ica_mixing.tsv",
        )
    
    def roi_stats(self):
        return op.join(
            self.projectdir,
            "GroupResults",
            "ROIstats",
        )

In [ ]:
if "biowulf" in socket.gethostname():
    basedir = "/data/"
else:
    basedir = "/Volumes/"

subject = 1
run = 3
component = 73

namer = DataFinder(basedir, subject, run)
Y = np.asarray(pd.read_csv(namer.mixing_matrix(), sep='\t'))
X_full = np.asarray(pd.read_csv(namer.full_model()))
betas = pd.read_csv(namer.combined_betas())

betas.drop(
    columns=[betas.columns[0], betas.columns[1]],
    axis=1,
    inplace=True,
)

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ica = Y
fit = np.asarray(np.matmul(X_full[:, 2:], betas.T))

c = component

ica_ts = ica[:, c]
fit_ts = fit[:, c]
ax.plot(ica_ts, color='black')
ax.plot(fit_ts, color='red')

In [ ]:
def scatter_cme(ax, fname: str, to_plot) -> None:
    df = pd.read_csv(fname)
    kappas = df["kappa"]
    rhos = df["rho"]
    varex = df["variance explained"]
    size = np.sqrt(varex) * 20
    rej_both = np.logical_and(df["Tedana Rejected"] == True, df["Regressors Rejected"] == True)
    rej_tedonly = np.logical_and(df["Tedana Rejected"] == True, df["Regressors Rejected"] == False)
    rej_regonly = np.logical_and(df["Tedana Rejected"] == False, df["Regressors Rejected"] == True)
    acc_all = np.logical_and(df["Tedana Rejected"] == False, df["Regressors Rejected"] == False)
    colors = pd.Series(data=["none" for _ in kappas])
    if to_plot[0]:
        colors[acc_all] = "green"
    if to_plot[1]:
        colors[rej_both] = "red"
    if to_plot[2]:
        colors[rej_tedonly] = "orange"
    if to_plot[3]:
        colors[rej_regonly] = "brown"
    ax.scatter(kappas, rhos, s=size, c=colors)

In [ ]:
for s in (np.arange(13) + 1):
    for r in (np.arange(3) + 1):
        namer = DataFinder(basedir, s, r)
        df = pd.read_csv(namer.combined_metrics())
        is_kappa_150 = df["kappa"] > 150
        idx = df["Component"][is_kappa_150]
        if len(idx) > 0:
            print(f"Subject {s} run {r} idx {idx}")

In [ ]:
df

In [ ]:
dfs = [
    pd.read_csv(op.join(namer.roi_stats(), f), sep='\t') for f in (
        "e2_mot_CSF_FisherZ_VisAud_group.tsv",
        "OC_mot_CSF_FisherZ_VisAud_group.tsv",
        "orthtedana_mot_csf_FisherZ_VisAud_group.tsv",
        "combined_regressors_FisherZ_VisAud_group.tsv"
    )
]

In [ ]:
rois=['L V1','R V1','L A1','R A1','lPrecuneus','rPrecuneus','lIFG','lSTS','rSTS','rMTG']

In [ ]:
data = [pd.melt(dfs[i][rois], var_name="FisherZ Visual-Audio") for i in range(len(dfs))]
print(len(data))

In [ ]:
glms = ["Echo 2 + Mot/CSF", "Combined + Mot/CSF", "Denoised + Mot/CSF", "Combined"]
for i in range(len(data)):
    data[i]["GLM"] = glms[i]

In [ ]:
merged = pd.concat(data, axis=0)

In [ ]:
merged.columns

In [ ]:
roi_means = pd.DataFrame(columns=("Mean Z-stat", "GLM"))
data_temp_ = np.zeros((len(rois), len(glms)), dtype=np.float32)
for i in range(len(rois)):
    for glm in range(len(glms)):
        data_temp_[i][glm] = np.mean(
                merged[
                    np.logical_and(
                        merged["FisherZ Visual-Audio"] == rois[i],
                        merged["GLM"] == glms[glm]
                    )
                ]["value"]
            )
        
glm_labels = [L + " Mean" for L in glms]
glm_temp_ = [None for _ in range(np.product(data_temp_.shape))]
for i in range(len(rois)):
    for j in range(len(glms)):
        glm_temp_[i*len(glms) + j] = glm_labels[j]

rois_super_temp_ = [[rois[i]]*4 for i in range(len(rois))]
rois_long = [item for sublist in rois_super_temp_ for item in sublist]
roi_means=pd.DataFrame(data=data_temp_.flatten(), columns=["value"])
roi_means["ROIS"] = rois_long
roi_means["GLM"] = glm_temp_
roi_means

In [ ]:
mean_labels = [L + " Mean" for L in glms]
use_labels = glms
for m in mean_labels:
    use_labels.append(m)

In [ ]:
sns_theme_dict = {
    'figure.facecolor': 'white',
    'axes.facecolor': 'white',
    'grid.color': 'white',
    'figure.figsize': (11.7,7)
}

In [ ]:
plt.rcParams.keys()

In [ ]:
sns.set_theme(rc=sns_theme_dict, font='Baskerville', font_scale=1.3)
plt.plot(np.linspace(-0.5, 9.5), np.zeros(50), 'k--', alpha=0.5)
fig = sns.stripplot(
    x='FisherZ Visual-Audio', y='value', data=merged, hue='GLM', dodge=0.4
)
p = sns.pointplot(
    x='ROIS', y="value", hue='GLM', data=roi_means, dodge=0.5, palette='dark', markers='d',
    join=False
)
fig.axes.set_xlabel("ROI")
fig.axes.set_ylabel("Fisher Z-Statistic Audio-Visual")
fig.legend(loc="upper right", fontsize=9)
plt.savefig("/Users/handwerkerd/Documents/Presentations/HBM2022/FisherZ_Visaud_plot.svg")

In [ ]:
dfs = [
    pd.read_csv(op.join(namer.roi_stats(), f), sep='\t') for f in (
        "e2_mot_CSF_FisherZ_WNW_group.tsv",
        "OC_mot_CSF_FisherZ_WNW_group.tsv",
        "orthtedana_mot_csf_FisherZ_WNW_group.tsv",
        "combined_regressors_FisherZ_WNW_group.tsv"
    )
]
data = [pd.melt(dfs[i][rois], var_name="FisherZ WNW") for i in range(len(dfs))]
glms = ["Echo 2 + Mot/CSF", "Combined + Mot/CSF", "Denoised + Mot/CSF", "Combined"]
for i in range(len(data)):
    data[i]["GLM"] = glms[i]
    
merged = pd.concat(data, axis=0)
roi_means = pd.DataFrame(columns=("Mean Z-stat", "GLM"))
data_temp_ = np.zeros((len(rois), len(glms)), dtype=np.float32)
for i in range(len(rois)):
    for glm in range(len(glms)):
        data_temp_[i][glm] = np.mean(
                merged[
                    np.logical_and(
                        merged["FisherZ WNW"] == rois[i],
                        merged["GLM"] == glms[glm]
                    )
                ]["value"]
            )
        
glm_labels = [L + " Mean" for L in glms]
glm_temp_ = [None for _ in range(np.product(data_temp_.shape))]
for i in range(len(rois)):
    for j in range(len(glms)):
        glm_temp_[i*len(glms) + j] = glm_labels[j]

rois_super_temp_ = [[rois[i]]*4 for i in range(len(rois))]
rois_long = [item for sublist in rois_super_temp_ for item in sublist]
roi_means=pd.DataFrame(data=data_temp_.flatten(), columns=["value"])
roi_means["ROIS"] = rois_long
roi_means["GLM"] = glm_temp_
plt.clf()
# sns.set(font_scale=1.4)
sns.set_theme(rc=sns_theme_dict, font='Baskerville', font_scale=1.3)

plt.plot(np.linspace(-0.5, 9.5), np.zeros(50), 'k--', alpha=0.5)
fig = sns.stripplot(
    x='FisherZ WNW', y='value', data=merged, hue='GLM', dodge=0.4
)

p = sns.pointplot(
    x='ROIS', y="value", hue='GLM', data=roi_means, dodge=0.5, palette='dark', markers='D',
    join=False, annot_kws={"size": 32}
)
fig.axes.set_xlabel("ROI")
fig.axes.set_ylabel("Fisher Z-Statistic Word-NonWord")
fig.legend(loc="best", fontsize=9)
plt.savefig("/Users/handwerkerd/Documents/Presentations/HBM2022/FisherZ_WNW_plot.svg")
